In [ ]:
# @title Complete Reset + Launch (Fixed)
import os
import threading
import time
import uvicorn
import gc
from google.colab.output import serve_kernel_port_as_window

# Clean everything
!rm -rf /content/Chatterbox-TTS-Server
!fuser -k 8004/tcp 2>/dev/null || echo "Port clear"
!pkill -f "uvicorn" 2>/dev/null || echo "No servers"

# Clear Hugging Face cache to avoid conflicts
!rm -rf /root/.cache/huggingface
print("🧹 Cleared HF cache to avoid token conflicts")

# Install system deps
!apt-get update -qq
!apt-get install -y build-essential cmake pkg-config libfftw3-dev libgsl-dev

# Clone fresh
!git clone https://github.com/devnen/Chatterbox-TTS-Server.git
%cd /content/Chatterbox-TTS-Server

print("✅ Fresh installation starting...")

# Install dependencies
!pip install torch==2.5.1+cu121 torchaudio==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121 -q
!pip install git+https://github.com/devnen/chatterbox.git -q
!pip install fastapi uvicorn pyyaml soundfile librosa safetensors -q
!pip install python-multipart requests jinja2 watchdog aiofiles pydub -q

# Create Swedish config
with open('config.yaml', 'w') as f:
    f.write("""server:
  host: "0.0.0.0"
  port: 8004

model:
  repo_id: "ResembleAI/chatterbox-multilingual"

tts_engine:
  device: "cuda"

generation_defaults:
  temperature: 0.3
  exaggeration: 0.4
  cfg_weight: 0.2
  speed_factor: 0.8
  language: "sv"

ui_state:
  split_text: true
  chunk_size: 200
  language: "sv"

ui:
  show_language_select: true
""")

print("✅ Installation complete, starting server...")

# Clear HF login if exists (avoid conflicts)
try:
    from huggingface_hub import logout
    logout()
    print("🔓 Logged out of HF to avoid conflicts")
except:
    pass

def run_server():
    try:
        os.chdir('/content/Chatterbox-TTS-Server')
        from server import app
        print("✅ Server module loaded successfully")
        uvicorn.run(app, host="0.0.0.0", port=8004, log_level="error")
    except Exception as e:
        print(f"❌ Server error: {e}")
        import traceback
        traceback.print_exc()

print("🚀 Starting server thread...")
thread = threading.Thread(target=run_server, daemon=True)
thread.start()

time.sleep(35)  # Extra time for model download

print("🇸🇪 Swedish TTS Server ready!")
serve_kernel_port_as_window(8004)  # FIXED function name

print("\n🎯 Test Swedish generation:")
print("Text: 'Hej, detta är svensk tal'")
print("Language: sv")

In [5]:
# @title 2. Launch the Server! ⚡
# Starts the TTS server with memory optimization

import threading
import time
import uvicorn
import gc
from google.colab.output import serve_kernel_port_as_window

port = 8004

print("🛑 Stopping existing servers...")
!fuser -k 8004/tcp 2>/dev/null || echo "Port clear"
!pkill -f "uvicorn.*server" 2>/dev/null || echo "No running servers"

print("🧹 Clearing memory...")
gc.collect()

%cd /content/Chatterbox-TTS-Server

def run_server():
    try:
        from server import app
        print("✅ Server starting...")
        # Add memory-efficient settings
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=port,
            log_level="warning",  # Reduce log verbosity
            access_log=False      # Disable access logs to save memory
        )
    except Exception as e:
        print(f"❌ Server error: {e}")

print("🚀 Starting TTS server...")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

time.sleep(25)  # Extra time for model loading

print("🎉 Server ready! Click below:")
serve_kernel_port_as_window(port)

print("\n💡 Tips to prevent crashes:")
print("• Use shorter texts (under 500 characters)")
print("• Wait for each generation to complete before starting another")
print("• If it crashes, just re-run this cell")

🛑 Stopping existing servers...
Port clear
^C
🧹 Clearing memory...
[Errno 2] No such file or directory: '/content/Chatterbox-TTS-Server'
/content
🚀 Starting TTS server...
❌ Server error: No module named 'server'
🎉 Server ready! Click below:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>


💡 Tips to prevent crashes:
• Use shorter texts (under 500 characters)
• Wait for each generation to complete before starting another
• If it crashes, just re-run this cell
